In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import PowerTransformer, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

RAND_STATE = 32 # for reproducible shuffling
TT_RATIO = 0.25 # test/train

from sklearn.linear_model import Lasso,Ridge,ElasticNet, LinearRegression

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
import warnings
warnings.filterwarnings('ignore')

In [2]:
data_london = pd.read_csv('data_london.csv')

In [3]:
data_london.corr()

,Unnamed: 0,realSum,person_capacity,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,metro_dist,attr_index_norm,rest_index_norm,room_shared_True,room_private_True,host_is_superhost_True,weekend_yes
Unnamed: 0,1.000000,0.029878,0.038673,-0.022818,0.019586,-0.037939,-0.040796,0.034566,-0.081320,-0.127053,0.035320,0.022382,0.011580,-0.020586,-0.041323,0.129947
realSum,0.029878,1.000000,0.352445,-0.046090,0.079923,0.014707,0.004005,0.312007,-0.217527,-0.150859,0.265354,0.245645,-0.027480,-0.337497,-0.014281,0.004399
person_capacity,0.038673,0.352445,1.000000,-0.050237,0.185592,-0.077125,-0.085755,0.576223,-0.115907,-0.076363,0.123346,0.130671,-0.009457,-0.547766,-0.078290,0.010201
multi,-0.022818,-0.046090,-0.050237,1.000000,-0.490243,0.058433,0.092912,0.017824,0.087984,0.082706,-0.087319,-0.090198,-0.002381,0.130059,0.145500,0.011605
biz,0.019586,0.079923,0.185592,-0.490243,1.000000,-0.190448,-0.288884,-0.042953,-0.270078,-0.249492,0.274595,0.277275,-0.015699,-0.149300,-0.172627,-0.066473
cleanliness_rating,-0.037939,0.014707,-0.077125,0.058433,-0.190448,1.000000,0.754588,0.033284,0.056256,0.054480,-0.032352,-0.040377,-0.015508,0.041809,0.224249,0.017481
guest_satisfaction_overall,-0.040796,0.004005,-0.085755,0.092912,-0.288884,0.754588,1.000000,0.057077,0.085487,0.094351,-0.066877,-0.069193,-0.011987,0.099184,0.228185,0.025846
bedrooms,0.034566,0.312007,0.576223,0.017824,-0.042953,0.033284,0.057077,1.000000,0.019334,0.047825,0.010161,0.000352,-0.015761,-0.207155,0.011103,0.008050
dist,-0.081320,-0.217527,-0.115907,0.087984,-0.270078,0.056256,0.085487,0.019334,1.000000,0.693476,-0.801185,-0.749932,0.028024,0.176972,0.050166,0.000475
metro_dist,-0.127053,-0.150859,-0.076363,0.082706,-0.249492,0.054480,0.094351,0.047825,0.693476,1.000000,-0.437266,-0.424800,0.043615,0.151233,0.039970,0.009389


In [4]:
data_london = data_london.drop(['Unnamed: 0', 'cleanliness_rating', 'host_is_superhost_True', 'weekend_yes', 'multi'], axis=1)

In [5]:
x = data_london.drop('realSum', axis=1)
y = data_london.realSum

In [6]:
# test-train split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=TT_RATIO, random_state=RAND_STATE)
x_train = pd.DataFrame(x_train)
x_test = pd.DataFrame(x_test)
x_train.describe()

,person_capacity,biz,guest_satisfaction_overall,bedrooms,dist,metro_dist,attr_index_norm,rest_index_norm,room_shared_True,room_private_True
count,7494.000000,7494.000000,7494.000000,7494.000000,7494.000000,7494.000000,7494.000000,7494.000000,7494.000000,7494.000000
mean,2.850147,0.388177,90.593408,1.132239,5.328843,1.006254,20.525015,11.219337,0.004670,0.557780
std,1.244913,0.487368,11.423167,0.578768,2.717192,1.268007,11.856743,6.911215,0.068185,0.496683
min,2.000000,0.000000,20.000000,0.000000,0.130000,0.010000,4.778228,2.515094,0.000000,0.000000
25%,2.000000,0.000000,86.000000,1.000000,3.540000,0.320000,12.305094,6.839707,0.000000,0.000000
50%,2.000000,0.000000,94.000000,1.000000,4.910000,0.530000,17.394838,9.495548,0.000000,1.000000
75%,4.000000,1.000000,99.000000,1.000000,6.860000,1.060000,25.097113,13.795697,0.000000,1.000000
max,6.000000,1.000000,100.000000,8.000000,17.320000,9.170000,100.000000,100.000000,1.000000,1.000000


In [7]:
x_train_const = sm.add_constant(x_train) # adding a constant

model = sm.OLS(y_train, x_train_const).fit()
predictions_train = model.predict(x_train_const)

x_test_const = sm.add_constant(x_test) # adding a constant
predictions_test = model.predict(x_test_const)
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                realSum   R-squared:                       0.207
Model:                            OLS   Adj. R-squared:                  0.206
Method:                 Least Squares   F-statistic:                     195.6
Date:                Mon, 06 Mar 2023   Prob (F-statistic):               0.00
Time:                        17:22:14   Log-Likelihood:                -56366.
No. Observations:                7494   AIC:                         1.128e+05
Df Residuals:                    7483   BIC:                         1.128e+05
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const               

In [8]:
sts = StandardScaler()

x_train_standarized = sts.fit_transform(x_train)
x_test_standarized = sts.transform(x_test)

x_train_standarized = pd.DataFrame(x_train_standarized)

In [9]:
x_train_const = sm.add_constant(x_train_standarized.to_numpy()) # adding a constant

model = sm.OLS(y_train, x_train_const).fit()
predictions_train = model.predict(x_train_const)

x_test_const = sm.add_constant(x_test_standarized) # adding a constant
predictions_test = model.predict(x_test_const)
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                realSum   R-squared:                       0.207
Model:                            OLS   Adj. R-squared:                  0.206
Method:                 Least Squares   F-statistic:                     195.6
Date:                Mon, 06 Mar 2023   Prob (F-statistic):               0.00
Time:                        17:22:14   Log-Likelihood:                -56366.
No. Observations:                7494   AIC:                         1.128e+05
Df Residuals:                    7483   BIC:                         1.128e+05
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        366.8811      5.167     71.000      0.0

In [10]:
lm = LinearRegression()
model = lm.fit(x_train,y_train)
model.coef_
model.intercept_

-140.65954857381308

In [11]:
model.score(x_train, y_train)

0.20723127033822597

In [12]:
y_pred = pd.DataFrame(model.predict(x_test),columns = ['realSum'] )      # model prediction
y_pred_train =  pd.DataFrame(model.predict(x_train),columns = ['realSum'])

In [13]:
mse(y_test,y_pred)

85071.27996640916

In [14]:
rmse = np.sqrt(mse(y_test,y_pred))
rmse

291.66981325877583

In [15]:
##prediction on the train set
mse(y_train,y_pred_train)

199803.83377698075

In [16]:
mae(y_test,y_pred)

123.03155960719138

In [17]:
R2=r2_score(y_test,y_pred)
R2

0.35710575459617966

In [18]:
R2_test=model.score(x_test,y_test)
R2_train=model.score(x_train,y_train)
Adj_R2= 1 - (1-R2)*(len(y_test)-1)/(len(y_test)-x_test.shape[1]-1)
Adj_R2

0.35452177451015143

In [19]:
model_lasso=Lasso(alpha=0.05)

model_lasso.fit(x_train, y_train)
print(f"{model_lasso.__class__.__name__}: Train -> {model_lasso.score(x_train, y_train)}, Test -> {model_lasso.score(x_test, y_test)}")

Lasso: Train -> 0.20722888555373897, Test -> 0.35706397648795807


In [20]:
model_ridge=Ridge(alpha=1000)
model_ridge.fit(x_train, y_train)
print(f"{model_ridge.__class__.__name__}: Train -> {model_ridge.score(x_train, y_train)}, Test -> {model_ridge.score(x_test, y_test)}")

Ridge: Train -> 0.19831057767824767, Test -> 0.34624971647131253


In [21]:
model = DecisionTreeRegressor()
model.fit(x_train, y_train)

print("X_train has %d rows and %d columns"  %(x_train.shape[0],x_train.shape[1]))
print("-----------------------------------")
print("The coefficient of determination for the test data is R2=%.2f"
      %(model.score(x_test, y_test)))
print("The coefficient of determination for the train data is R2=%.2f"
      %(model.score(x_train, y_train)))


X_train has 7494 rows and 10 columns
-----------------------------------
The coefficient of determination for the test data is R2=-0.76
The coefficient of determination for the train data is R2=1.00


In [22]:
scores=cross_val_score(model, x_train, y_train, cv=5)
print("Cross validation scores: ", scores)
print("Score stats: %0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

Cross validation scores:  [-1.15878829  0.11317574 -1.40236837  0.24685051 -0.18982191]
Score stats: -0.48 accuracy with a standard deviation of 0.67


In [23]:
model1 = DecisionTreeRegressor()
model2 = LinearRegression()
model3 = KNeighborsRegressor()

model_pipeline = [model1, model2, model3]
model_names = ['Regression Tree', 'Linear Regression', 'KNN']
scores = {}
i=0
for model in model_pipeline:
    mean_score = np.mean(cross_val_score(model, x_train, y_train, cv=5))
    scores[model_names[i]] = mean_score
    i = i+1
print(scores)

{'Regression Tree': -0.1301650273419835, 'Linear Regression': 0.2901662156268346, 'KNN': 0.09581710562127337}
